In [1]:
# importing libraries
import os
from pathlib import Path
import pandas as pd 
import seaborn as sns
import urllib
import json
import zipfile
from zipfile import ZipFile
from openpyxl import Workbook
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
# clear folder from previous exercises leaving only trip data zip file
dir_name = r"C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data"
folder = os.listdir(dir_name)

for item in folder:
    if item.endswith(".csv") or item.endswith(".txt"):
        os.remove(os.path.join(dir_name, item))

In [3]:
# assign path to data folder
data_path = r"C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data"

# file name to download
file = "OpenDataDC_Taxi_2019.zip"

# establish file path
fullfilename = os.path.join(data_path, file)

# assign ZIP file to variable
items = zipfile.ZipFile(fullfilename)

# show files in ZIP
print(items.namelist())

# opening ZIP file in read mode
with zipfile.ZipFile(fullfilename, "r") as file:

    # extract all files inside the ZIP file
    file.extractall(data_path)

['taxi_2019_06.txt', 'taxi_2019_01.txt', 'taxi_2019_02.txt', 'taxi_2019_03.txt', 'taxi_2019_04.txt', 'taxi_2019_05.txt', 'README_DC_Taxicab_trip.txt', 'taxi_2019_07.csv', 'taxi_2019_08.csv', 'taxi_2019_09.csv', 'taxi_2019_10.csv', 'taxi_2019_11.csv', 'taxi_2019_12.csv']


In [ ]:
# convert txt files to csv (download folder contains both txt and csv files)
## data types indicated to improve performance
import fnmatch

data_path = r"C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data"
files = os.listdir(data_path)

for file in files:
    if fnmatch.fnmatch(file, "taxi*"):
        extension = os.path.splitext(file)[1]
        if extension == ".txt":
            filename = os.path.join(data_path, file)
            df = pd.read_csv(
                filename,
                sep="|",
                dtype={
                    "OBJECTID": int,
                    "FAREAMOUNT": float,
                    "ORIGINZIP": object,
                    "DESTINATIONZIP": object,
                    "MILEAGE": float,
                    "DURATION": int,
                    "ORIGIN_BLOCK_LATITUDE": float,
                    "ORIGIN_BLOCK_LONGITUDE": float,
                    "ORIGINDATETIME_TR": object,
                    "DESTINATIONDATETIME_TR": object,
                }
            )
            new_filename = os.path.splitext(filename)[0] + ".csv"
            df.to_csv(new_filename, index=False)

In [5]:
# load monthly files into a single df and visualize process using glob library
## data types indicated to improve performance and columns selected to consolidate df
import glob

all_dfs = []
for one_filename in glob.glob(
    r"C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data\taxi*.csv"
):
    print(f"Loading {one_filename}")
    new_df = pd.read_csv(
        one_filename,
        dtype={
            "OBJECTID": int,
            "FAREAMOUNT": float,
            "ORIGINZIP": object,
            "DESTINATIONZIP": object,
            "MILEAGE": float,
            "DURATION": int,
            "ORIGIN_BLOCK_LATITUDE": float,
            "ORIGIN_BLOCK_LONGITUDE": float,
            "ORIGINDATETIME_TR": object,
            "DESTINATIONDATETIME_TR": object,
        },
        usecols=[
            "OBJECTID",
            "ORIGINDATETIME_TR",
            "DESTINATIONDATETIME_TR",
            "ORIGINZIP",
            "DESTINATIONZIP",
            "MILEAGE",
            "DURATION",
            "FAREAMOUNT",
            "ORIGIN_BLOCK_LATITUDE",
            "ORIGIN_BLOCK_LONGITUDE",
            "ORIGINZIP"
        ]
    )
    all_dfs.append(new_df)

len(all_dfs)

Loading C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data\taxi_2019_01.csv
Loading C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data\taxi_2019_02.csv
Loading C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data\taxi_2019_03.csv
Loading C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data\taxi_2019_04.csv
Loading C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data\taxi_2019_05.csv
Loading C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Documents\Projects_JM\Python\Projects\taxi_analysis\data\taxi_2019_06.csv
Loading C:\Users\Guest.Guest\OneDrive - Government of The District of Columbia\Doc

12

In [6]:
# combine multiple dataframes into one and preview data
taxi_trips = pd.concat(all_dfs)

rows, columns = taxi_trips.shape
print('rows: {:,}\ncolumns: {}'.format(rows, columns))

taxi_trips

rows: 8,652,721
columns: 10


,OBJECTID,FAREAMOUNT,ORIGINZIP,DESTINATIONZIP,MILEAGE,DURATION,ORIGIN_BLOCK_LATITUDE,ORIGIN_BLOCK_LONGITUDE,ORIGINDATETIME_TR,DESTINATIONDATETIME_TR
0,48252809,15.94,20006,22202,5.19,879,38.900213,-77.040574,01/01/2019 15:00,01/01/2019 15:00
1,48252810,73.45,20007,20152,31.43,2735,38.907217,-77.063241,01/01/2019 15:00,01/01/2019 16:00
2,48252811,6.49,20037,20007,1.23,363,38.909637,-77.047716,01/01/2019 15:00,01/01/2019 15:00
3,48252812,17.29,20009,22202,6.13,946,38.920843,-77.031850,01/01/2019 15:00,01/01/2019 15:00
4,48252813,45.37,-,22202,0.00,0,NaN,NaN,01/01/2019 15:00,01/01/2019 15:00
...,...,...,...,...,...,...,...,...,...,...
574395,574396,13.78,20002,20011,5.25,0,38.911114,-77.005668,12/31/2019 23:00,12/31/2019 20:00
574396,574397,12.70,20782,20011,4.30,0,NaN,NaN,12/31/2019 23:00,12/31/2019 17:00
574397,574398,8.38,-,-,3.13,0,38.931227,-77.013912,12/31/2019 23:00,12/31/2019 23:00
574398,574399,6.76,-,-,0.41,0,38.879633,-77.024955,12/31/2019 23:00,12/31/2019 23:00


In [7]:
# find missing values, shown as percentage
print((taxi_trips.isnull().sum()/len(taxi_trips.index))*100)

OBJECTID                   0.000000
FAREAMOUNT                 1.884448
ORIGINZIP                  0.000000
DESTINATIONZIP             0.000000
MILEAGE                    0.000000
DURATION                   0.000000
ORIGIN_BLOCK_LATITUDE     15.880993
ORIGIN_BLOCK_LONGITUDE    15.880993
ORIGINDATETIME_TR          0.000000
DESTINATIONDATETIME_TR     0.000000
dtype: float64


In [8]:
# drop records with NA values for columns that will be used for analysis
taxi_trips.dropna(subset=['ORIGIN_BLOCK_LATITUDE', 'ORIGIN_BLOCK_LONGITUDE'], inplace=True)

In [9]:
# confirm rows with missing values were dropped
print((taxi_trips.isnull().sum()/len(taxi_trips.index))*100)

OBJECTID                  0.000000
FAREAMOUNT                1.784084
ORIGINZIP                 0.000000
DESTINATIONZIP            0.000000
MILEAGE                   0.000000
DURATION                  0.000000
ORIGIN_BLOCK_LATITUDE     0.000000
ORIGIN_BLOCK_LONGITUDE    0.000000
ORIGINDATETIME_TR         0.000000
DESTINATIONDATETIME_TR    0.000000
dtype: float64
